# Getting Data

In [39]:
#!kaggle competitions download -c store-sales-time-series-forecasting

In [40]:
import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [41]:
data=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv", parse_dates=['date'], index_col=['date'])
data.head()

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0
2013-01-01,2,1,BEAUTY,0.0,0
2013-01-01,3,1,BEVERAGES,0.0,0
2013-01-01,4,1,BOOKS,0.0,0


In [42]:
data.tail()

,id,store_nbr,family,sales,onpromotion
date,,,,,
2017-08-15,3000883,9,POULTRY,438.133,0
2017-08-15,3000884,9,PREPARED FOODS,154.553,1
2017-08-15,3000885,9,PRODUCE,2419.729,148
2017-08-15,3000886,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
2017-08-15,3000887,9,SEAFOOD,16.000,0


In [43]:
submit=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv", parse_dates=['date'], index_col=['date'])
submit.head()

,id,store_nbr,family,onpromotion
date,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0
2017-08-16,3000889,1,BABY CARE,0
2017-08-16,3000890,1,BEAUTY,2
2017-08-16,3000891,1,BEVERAGES,20
2017-08-16,3000892,1,BOOKS,0


In [44]:
submit_sample=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")
submit_sample.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


In [45]:
stores=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
print(stores.shape)
stores.head()

(54, 5)


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [46]:
oil=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv", parse_dates=['date'], index_col=['date'])
oil.head()

,dcoilwtico
date,
2013-01-01,NaN
2013-01-02,93.14
2013-01-03,92.97
2013-01-04,93.12
2013-01-07,93.20


In [47]:
holidays_events=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",parse_dates=['date'], index_col=['date'])
holidays_events.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [48]:
transactions=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv",parse_dates=['date'], index_col=['date'])
transactions.head()

,store_nbr,transactions
date,,
2013-01-01,25,770
2013-01-02,1,2111
2013-01-02,2,2358
2013-01-02,3,3487
2013-01-02,4,1922


# Missing Values

In [49]:
data.isna().sum()

id             0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [50]:
stores.isna().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [51]:
oil.isna().sum()

dcoilwtico    43
dtype: int64

In [52]:
oil[oil['dcoilwtico'].isna()]

,dcoilwtico
date,
2013-01-01,NaN
2013-01-21,NaN
2013-02-18,NaN
2013-03-29,NaN
2013-05-27,NaN
2013-07-04,NaN
2013-09-02,NaN
2013-11-28,NaN
2013-12-25,NaN


random values are missing. there are no values missing for a consecutive group of days. therefore, it is okay to replace each missing value with previous value. first value on index 0 can be filled by value at index 1

In [53]:
oil=oil.fillna(method="ffill")
oil=oil.fillna(method="bfill") #for 1st row

In [54]:
oil.isna().sum()

dcoilwtico    0
dtype: int64

In [55]:
holidays_events.isna().sum()

type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

# Plotting Time Series

In [56]:
data.describe(include='all')

,id,store_nbr,family,sales,onpromotion
count,3.000888e+06,3.000888e+06,3000888,3.000888e+06,3.000888e+06
unique,NaN,NaN,33,NaN,NaN
top,NaN,NaN,AUTOMOTIVE,NaN,NaN
freq,NaN,NaN,90936,NaN,NaN
mean,1.500444e+06,2.750000e+01,NaN,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,NaN,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,NaN,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,NaN,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,NaN,1.958473e+02,0.000000e+00


In [57]:
#temp=data
#temp = temp.groupby('family').resample('M').sum()
#fig = px.line(temp, x=temp.index.get_level_values('date'), 
#              y="sales", 
#              color=temp.index.get_level_values('family'))
#fig.show()

In [58]:
#temp=data
#temp = temp.groupby('store_nbr').resample('M').sum()
#fig = px.line(temp, x=temp.index.get_level_values('date'), 
#              y="sales", 
#              color=temp.index.get_level_values('store_nbr'))
#fig.show()

In [59]:
temp=data
temp = temp.resample('M').sum()
#temp.drop(temp.tail(1).index,inplace=True) # drop last month as only half of month data is given
fig = px.line(temp, x=temp.index, 
              y="sales")
fig.show()

In [60]:
temp1=data
temp1 = temp1.resample("M").sum()

temp2=oil
temp2 = temp2.resample('M').mean()



# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=temp1.index, y=temp1.sales, name="monthly sales"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=temp2.index, y=temp2.dcoilwtico, name="oil price"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="Months")

# Set y-axes titles
fig.update_yaxes(title_text="Monthly Sales", secondary_y=False)
fig.update_yaxes(title_text="Oil Price", secondary_y=True)

fig.show()

# Data Merging

In [61]:
print(data.shape)
data.head()

(3000888, 5)


,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0
2013-01-01,2,1,BEAUTY,0.0,0
2013-01-01,3,1,BEVERAGES,0.0,0
2013-01-01,4,1,BOOKS,0.0,0


In [62]:
print(stores.shape)
stores.head()

(54, 5)


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [63]:
data = data.reset_index().merge(stores, how="left").set_index('date')
print(data.shape)
data.head()

(3000888, 9)


,id,store_nbr,family,sales,onpromotion,city,state,type,cluster
date,,,,,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
2013-01-01,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2013-01-01,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
2013-01-01,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
2013-01-01,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13


In [64]:
oil.head()

,dcoilwtico
date,
2013-01-01,93.14
2013-01-02,93.14
2013-01-03,92.97
2013-01-04,93.12
2013-01-07,93.20


In [65]:
data = pd.merge(data, oil, left_index=True, right_index=True, how='left')
print(data.shape)
data.head()

(3000888, 10)


,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
date,,,,,,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14
2013-01-01,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14
2013-01-01,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14
2013-01-01,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14
2013-01-01,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14


In [66]:
temp=holidays_events
temp = temp[~temp.index.duplicated(keep='first')]
temp.rename(columns = {'type':'holiday_type'}, inplace = True)
data = pd.merge(data, temp, left_index=True, right_index=True, how='left')
print(data.shape)
data.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(3000888, 15)


,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,holiday_type,locale,locale_name,description,transferred
date,,,,,,,,,,,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False


In [67]:
transactions.head()

,store_nbr,transactions
date,,
2013-01-01,25,770
2013-01-02,1,2111
2013-01-02,2,2358
2013-01-02,3,3487
2013-01-02,4,1922


In [68]:
temp=transactions
#temp = temp[~temp.index.duplicated(keep='first')]
#temp.rename(columns = {'store_nbr':'sn_trans'}, inplace = True)
data = pd.merge(data, temp, left_on=[data.index, 'store_nbr'], right_on=[temp.index, 'store_nbr'], how='left')
#temp = pd.merge(data, temp, left_index=True, right_index=True, how='left')
#temp.loc[temp['store_nbr'] != temp['sn_trans'], 'transactions'] = 0
#temp.drop('sn_trans')
del temp
print(data.shape)
data.head()

(3000888, 17)


,key_0,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,holiday_type,locale,locale_name,description,transferred,transactions
0,2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN
1,2013-01-01,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN
2,2013-01-01,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN
3,2013-01-01,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN
4,2013-01-01,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN


In [69]:
data.index = pd.to_datetime(data.index)

data['day_of_week'] = data.index.day_name()
data.head()

,key_0,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,holiday_type,locale,locale_name,description,transferred,transactions,day_of_week
1970-01-01 00:00:00.000000000,2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday
1970-01-01 00:00:00.000000001,2013-01-01,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday
1970-01-01 00:00:00.000000002,2013-01-01,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday
1970-01-01 00:00:00.000000003,2013-01-01,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday
1970-01-01 00:00:00.000000004,2013-01-01,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday


In [70]:
data['day_of_week'].unique()

array(['Thursday'], dtype=object)

In [72]:
data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day
print(data.shape)
data.head()

(3000888, 21)


,key_0,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_type,locale,locale_name,description,transferred,transactions,day_of_week,year,month,day
1970-01-01 00:00:00.000000000,2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday,1970,1,1
1970-01-01 00:00:00.000000001,2013-01-01,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday,1970,1,1
1970-01-01 00:00:00.000000002,2013-01-01,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday,1970,1,1
1970-01-01 00:00:00.000000003,2013-01-01,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday,1970,1,1
1970-01-01 00:00:00.000000004,2013-01-01,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Thursday,1970,1,1


In [73]:
data.shape

(3000888, 21)

In [ ]:
#data = pd.get_dummies(data)
#print(data.shape)
#data.head()